In [1]:
import scipy.stats as sps
from sklearn.model_selection import train_test_split
import gc
from dateutil.parser import *
import pandas as pd
import numpy as np
import os
import yt.wrapper as yt 
os.chdir('/Users/lunin-dv/Desktop/Library/')
import importlib
import my_library as lib
import operator
import re
importlib.reload(lib)
os.chdir('/Users/lunin-dv/Desktop/data/')

In [2]:
def copy_before(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) == 0:
        yt.copy("//home/cloud_analytics/emailing/sender_copy/" + table_name,
                "//home/cloud_analytics/emailing/sender/" + table_name,
                force=True)
        print('saved')
copy_before("BA-Activation-Program-Events")
copy_before("BA-Activation-Program-Events-OLD")

## Unsubscribed

In [19]:
req = """
SELECT
    DISTINCT email
FROM "//home/cloud_analytics/export/marketo/ya_fill_out_form"
WHERE webform_id == '10014482'
FORMAT TabSeparatedWithNames
"""
unsubscribed_emails = lib.execute_query(req)
unsubscribed_emails = set(unsubscribed_emails['email'])

## OLD 

In [20]:
req = """
SELECT
DISTINCT
    email
FROM "//home/cloud_analytics/cubes/emailing_events/emailing_events"
WHERE event == 'add_to_nurture_stream'
AND lower(program_name) like '%ba-activation-program%'
FORMAT TabSeparatedWithNames
"""

old_programm_df = lib.execute_query(req)

In [21]:
req = """
SELECT
    DISTINCT
        multiIf(
            email LIKE '%@yandex.%' OR email LIKE '%@ya.%',
            CONCAT(lower(replaceAll(splitByString('@',assumeNotNull(email))[1], '.', '-')), '@yandex.ru'),
            lower(email)
        ) as email,
        modulo(sipHash64(email), 100) as group_key,
        if (group_key < 35 or group_key > 44, 'test', 'control') as Group,
        event_name,
        Max(event_link) as event_link,
        MAx(billing_account_id) as billing_account_id,
        MAX(console_registration_date) as console_registration_date,
        MAX(ba_created_at) as ba_created_at,
        MAX(mail_marketing) as mail_marketing,
        MAX(mail_feature) as mail_feature,
        MAX(mail_info) as mail_info,
        MAX(mail_promo) as mail_promo,
        MAX(mail_event) as mail_event
FROM "//home/cloud_analytics/export/marketo/ya_attend_event" as a
ANY LEFT JOIN (
    SELECT
        dwh_id,
        argMax(billing_account_id, dwh_updated_at) as billing_account_id,
        argMax(console_registration_date, dwh_updated_at) as console_registration_date,
        argMax(ba_created_at, dwh_updated_at) as ba_created_at,
        argMax(mail_marketing, dwh_updated_at) as mail_marketing,
        argMax(mail_feature, dwh_updated_at) as mail_feature,
        argMax(mail_info, dwh_updated_at) as mail_info,
        argMax(mail_promo, dwh_updated_at) as mail_promo,
        argMax(mail_event, dwh_updated_at) as mail_event
    FROM "//home/cloud_analytics/export/marketo/lead"
    GROUP BY dwh_id
) as b
ON a.lead_dwh_id == b.dwh_id
WHERE isNotNull(email)
AND email != ''
GROUP BY email, group_key, Group, event_name
FORMAT TabSeparatedWithNames
"""
new_df = lib.execute_query(req)

In [22]:
new_df['was_in_marketo_ba_activation_programm'] = new_df['email'].isin(old_programm_df['email']).astype(int)

In [23]:
new_df['is_unsubscribed'] = new_df['email'].isin(unsubscribed_emails).astype(int)

In [25]:
def update_current_table(table_name, path, tables_to_update):
    full_path = path + "/" + table_name
    full_path_old = path + "/" + table_name + '-OLD'
    req = f"""
    SELECT
        *
    FROM "{full_path}"
    FORMAT TabSeparatedWithNames
    """
    old_df = lib.execute_query(req)
    if 'sended_mails' in old_df.columns:
        old_df['sended_mails'] = old_df['sended_mails'].apply(
            lambda x: x.replace("\\", ""))
    for table, key in tables_to_update:
        old_df = pd.merge(old_df,
                          table, on=key, suffixes=('', '_new'), how='left')
        
    new_cols = []
    for column in old_df.columns:
        if "_new" == column[-4:]:
            new_cols.append(column)
            old_column = column[:-4]
            old_df[old_column] = old_df[[old_column, column]].apply(
            lambda x: x[column] if not pd.isnull(x[column]) else x[old_column], axis=1
            )
            
    test_emails_before = lib.execute_query(f"""
    SELECT
        email
    FROM "{full_path_old}"
    FORMAT TabSeparatedWithNames
    """)
    test_emails_before = set(test_emails_before['email'])
    old_df_test_emails = set(old_df[old_df['Group'].str.lower() == 'test']['email'])
    old_df_test_emails.update(test_emails_before)
    old_df['Group'] = old_df['email'].apply(lambda x: 'test' if x in old_df_test_emails else 'control')
    old_df['group_key'] = old_df['email'].apply(lambda x: 1 if x in old_df_test_emails else 40)
    old_df.drop(columns=new_cols, inplace=True)
    lib.save_table(table_name, path, old_df)
    #return old_df, old_df_test_emails

In [28]:
tables_to_update = [(new_df, ["email", "event_name"])]

In [29]:
update_current_table("BA-Activation-Program-Events", "//home/cloud_analytics/emailing/sender", 
                     tables_to_update)

In [11]:
old_df = lib.execute_query("""
SELECT
    *
FROM "//home/cloud_analytics/emailing/sender/BA-Activation-Program-Events"
FORMAT TabSeparatedWithNames
""")

In [12]:
new_df = new_df[~new_df[['email', 'event_name']]\
                .apply(lambda x: x['email'] + "__" + x['event_name'], axis=1).isin(
                old_df[['email', 'event_name']].apply(lambda x: x['email'] + "__" + x['event_name'], axis=1))
               ]

In [13]:
new_df['experiment_date'] = lib.get_current_date_as_str()

In [14]:
assert set(new_df.columns) == set(old_df.columns)

In [18]:
lib.save_table("BA-Activation-Program-Events", 
               "//home/cloud_analytics/emailing/sender",
               new_df, append=True)

In [19]:
def copy_after(table_name):
    if yt.row_count("//home/cloud_analytics/emailing/sender/" + table_name) != 0:
        yt.copy("//home/cloud_analytics/emailing/sender/" + table_name,
                "//home/cloud_analytics/emailing/sender_copy/" + table_name, force=True)
copy_after("BA-Activation-Program-Events")
copy_after("BA-Activation-Program-Events-OLD")

In [19]:
# marketo_previous_req = """
# SELECT
#     DISTINCT
#     multiIf(
#             email LIKE '%@yandex.%' OR email LIKE '%@ya.%',
#             CONCAT(lower(replaceAll(splitByString('@',assumeNotNull(email))[1], '.', '-')), '@yandex.ru'),
#             lower(email)
#         ) as email,
#     groupUniqArray(mailing_name) as sended_mails,
#     'Test' as Group,
#     any(billing_account_id) as billing_account_id,
#     any(console_registration_date) as console_registration_date,
#     any(ba_created_at) as ba_created_at,
#     any(mail_marketing) as mail_marketing,
#     any(mail_feature) as mail_feature,
#     any(mail_info) as mail_info,
#     any(mail_promo) as mail_promo,
#     any(mail_event) as mail_event
# FROM "//home/cloud_analytics/cubes/emailing_events/emailing_events" as a
# ANY LEFT JOIN (
#     SELECT
#         multiIf(
#             email LIKE '%@yandex.%' OR email LIKE '%@ya.%',
#             CONCAT(lower(replaceAll(splitByString('@',assumeNotNull(email))[1], '.', '-')), '@yandex.ru'),
#             lower(email)
#         ) as email,
#         Max(billing_account_id) as billing_account_id,
#         Max(console_registration_date) as console_registration_date,
#         Max(ba_created_at) as ba_created_at,
#         Max(mail_marketing) as mail_marketing,
#         Max(mail_feature) as mail_feature,
#         Max(mail_info) as mail_info,
#         Max(mail_promo) as mail_promo,
#         Max(mail_event) as mail_event
#     FROM "//home/cloud_analytics/export/marketo/lead"
#     GROUP BY email
# ) as b
# ON a.email == b.email
# WHERE lower(mailing_name) like '%ba-activation-program-events%'
# GROUP BY email
# FORMAT TabSeparatedWithNames
# """
# marketo_prev_df = lib.execute_query(marketo_previous_req)
# marketo_prev_df['is_unsubscribed'] = marketo_prev_df['email'].isin(unsubscribed_emails).astype(int)
# marketo_prev_df['sended_mails'] = marketo_prev_df['sended_mails'].astype(str)
# lib.save_table("BA-Activation-Program-Events-OLD", "//home/cloud_analytics/emailing/sender", 
#                marketo_prev_df)